# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('../Orders.csv', encoding = "ISO-8859-1")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
orders.columns

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')

SUB PROBLEM_1 : How to aggregate the  amount_spent for unique customers?
groupby CUSTOMER ID que es comun, y sumar todos los amount_spent
osea, groupby + aggregate sum:

columns_c = ['CustomerID','amount_spent']
df = orders[columns_c]

agrupado = df.groupby('CustomerID')
agrupado.head()

suma = df.groupby('CustomerID').sum()
suma.head()

a = suma.sort_values('amount_spent', axis=0, ascending=False).head(10)
display(a)

In [4]:
# SUB PROBLEM_1 : How to aggregate the  amount_spent for unique customers?
    # groupby CUSTOMER ID que es comun, y sumar todos los amount_spent
    # osea, groupby + aggregate sum:
    
u_customers = orders.groupby('CustomerID').agg('sum')['amount_spent'].sort_values(ascending=False).reset_index()
u_customers.head()

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06


In [5]:
# SUB PROBLEM_2: How to select customers whose aggregated amount_spent is in a given quantile range?
# Podemos usar quantile(i) para obtenerlo
diez = u_customers['amount_spent'].quantile(0.1) # 10th percentile
setenta = u_customers['amount_spent'].quantile(0.75) # 75th percentile
noventa = u_customers['amount_spent'].quantile(0.95) # 90th percentile
print(diez,setenta,noventa)

preferred = u_customers[(u_customers['amount_spent']>=setenta)&(u_customers['amount_spent']<noventa)]
VIP = u_customers[(u_customers['amount_spent']>=noventa)]
display(preferred.head())
display(VIP.head())

156.56599999999997 1661.6400000000003 5840.181999999983


,CustomerID,amount_spent
217,13050,5836.86
218,12720,5781.73
219,15218,5756.89
220,17686,5739.46
221,13178,5725.47


,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06


In [6]:
# SUB PROBLEM_3: How to label selected customers as "VIP" or "Preferred"?

def getLabel(x):
    if x>=noventa:
        return 'VIP'
    elif x>=setenta and x<noventa:
        return 'preferred'
    else:
        return 'Regular'

u_customers['Classify'] = u_customers['amount_spent'].apply(getLabel)
#muestra:
u_customers[210:222]

,CustomerID,amount_spent,Classify
210,14101,5976.79,VIP
211,15382,5955.71,VIP
212,14505,5905.64,VIP
213,14415,5900.28,VIP
214,14944,5900.25,VIP
215,13209,5891.61,VIP
216,13534,5870.08,VIP
217,13050,5836.86,preferred
218,12720,5781.73,preferred
219,15218,5756.89,preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
# Uno customer ID and Country y elimino duplicados:
cus_count = orders[['CustomerID','Country']].drop_duplicates()
cus_count.head()


,CustomerID,Country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,France
46,13748,United Kingdom
65,15100,United Kingdom


In [8]:
u_customers[210:222]

,CustomerID,amount_spent,Classify
210,14101,5976.79,VIP
211,15382,5955.71,VIP
212,14505,5905.64,VIP
213,14415,5900.28,VIP
214,14944,5900.25,VIP
215,13209,5891.61,VIP
216,13534,5870.08,VIP
217,13050,5836.86,preferred
218,12720,5781.73,preferred
219,15218,5756.89,preferred


In [9]:
countries_classify = u_customers.merge(cus_count[['CustomerID','Country']], how='left', on='CustomerID')
countries_classify.drop_duplicates(inplace=True)
countries_classify.head()


# DUDAS PARA TERMINARLO

,CustomerID,amount_spent,Classify,Country
0,14646,280206.02,VIP,Netherlands
1,18102,259657.30,VIP,United Kingdom
2,17450,194550.79,VIP,United Kingdom
3,16446,168472.50,VIP,United Kingdom
4,14911,143825.06,VIP,EIRE


In [10]:
countries_classify_VIPS = countries_classify[countries_classify['Classify'] == 'VIP']
#most_VIP_countries.Classify.value_counts()

countries_classify_VIPS[['Country', 'CustomerID']].groupby(by='Country').count().sort_values(by='CustomerID', 
                                                                                        ascending=False)

,CustomerID
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2
Portugal,2
Australia,2
EIRE,2
Japan,2


In [12]:

countries_classify_VIPS.groupby(by='Country').agg({'CustomerID': 'count', 'amount_spent': 'max'})

,CustomerID,amount_spent
Country,,
Australia,2,124914.53
Belgium,1,6487.45
Channel Islands,1,8137.02
Cyprus,1,6372.58
Denmark,1,7829.89
EIRE,2,143825.06
Finland,1,7956.46
France,9,18895.91
Germany,10,19824.05


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [15]:
# Preferred Customers whose aggregated expenses are between the 75th and 95th percentile

# countries_classify_VIPS_Pref = countries_classify[(countries_classify['Classify'] == 'VIP') | (countries_classify['Classify'] == 'Preferred')]
countries_classify_VIPS_Pref = countries_classify[countries_classify['Classify'].isin(['VIP', 'preferred']) ]
countries_classify_VIPS_Pref.Classify.value_counts()



preferred    873
VIP          218
Name: Classify, dtype: int64

In [17]:
countries_classify_VIPS_Pref.groupby(by='Country').agg({'CustomerID': 'count'}).sort_

,CustomerID
Country,
Australia,4
Austria,3
Belgium,12
Canada,1
Channel Islands,4
Cyprus,4
Denmark,3
EIRE,3
Finland,5
